<a href="https://colab.research.google.com/github/Rajshekhar-Reddy1/-Insaid-AI-Regards/blob/main/NLP2_Module1_Assignment_Solutions_ipynb_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://github.com/insaid2018/Term-1/blob/master/Images/INSAID_Full%20Logo.png?raw=true" width="240" height="100" /></center>

**<center><h3>NLP2 Module 1 Assignment Solutions</h3></center>**

---
# **Table of Contents**
---

**1.** [**Dataset Description**](#Section1)<br>
**2.** [**Importing Libraries**](#Section2)<br>
**3.** [**Importing the Dataset**](#Section3)<br>
**4.** [**Data Pre-processing**](#Section4)<br>
**5.** [**Building LSTM Model**](#Section5)<br>
**6.** [**Generating Text with LSTM Network**](#Section6)<br>
**7.** [**Data Post-Processing**](#Section7)<br>
**8.** [**Building Larger LSTM Network**](#Section8)<br>
**9.** [**Generating Text**](#Section9)<br>
**10.** [**Conclusion**](#Section9)<br>

---
<a name = Section1></a>
# **1. Dataset Description**
---

 - We are going to use __The Adventures of Sherlock Holmes__ as the dataset.

<center><img src="https://raw.githubusercontent.com/insaid2018/DeepLearning/master/images/Adventures_of_sherlock_holmes.jpg" /></center>

---
<a name = Section2></a>
# **2. Importing Libraries**
---

In [ ]:
# Import tensorflow 2.x
# This code block will only work in Google Colab.
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical

In [ ]:
import numpy as np
import urllib
import sys

---
<a name = Section3></a>
# **3. Importing the Dataset**
---

In [ ]:
# Importing the dataset from github.
response = urllib.request.urlopen('https://raw.githubusercontent.com/insaid2018/DeepLearning/master/Data/The%20Adventures%20of%20Sherlock%20Holmes.txt')

**<h4>Question 1:** Complete the function to read the text from the `response` object.

<details>

**<summary>Hint:</summary>**

- Use the `read` method of the `response` object and **decode** the text using **utf8**.

- Convert the entire text to **lower** case.

</details>

In [ ]:
def read_text(response):
    # Read the text and decode it.
    raw_text = response.read().decode('utf8')

    # Convert the entire text to lower case.
    raw_text = raw_text.lower()

    return raw_text

In [ ]:
raw_text = read_text(response)

---
<a name = Section4></a>
# **4. Data Pre-processing**
---

In [ ]:
print(raw_text[:1000])

i. a scandal in bohemia

to sherlock holmes she is always _the_ woman. i have seldom heard him
mention her under any other name. in his eyes she eclipses and
predominates the whole of her sex. it was not that he felt any emotion
akin to love for irene adler. all emotions, and that one particularly,
were abhorrent to his cold, precise but admirably balanced mind. he
was, i take it, the most perfect reasoning and observing machine that
the world has seen, but as a lover he would have placed himself in a
false position. he never spoke of the softer passions, save with a gibe
and a sneer. they were admirable things for the observer—excellent for
drawing the veil from men’s motives and actions. but for the trained
reasoner to admit such intrusions into his own delicate and finely
adjusted temperament was to introduce a distracting factor which might
throw a doubt upon all his mental results. grit in a sensitive
instrument, or a crack in one of his own high-power lenses, would


**<h4>Question 2:** Create mapping of unique chars to integers.


<details>

**<summary>Hint:</summary>**

- Create a dictionary with the mapping, where **character** should be the **key** and it's **index** should be the **value**.

- Perform the operation on the **chars** list.

</details>

In [ ]:
chars = sorted(list(set(raw_text)))

In [ ]:
chars[:20]

['\n',
 '\r',
 ' ',
 '!',
 '&',
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [ ]:
def char_mapping(chars):
    char_to_int = dict((c, i) for i, c in enumerate(chars))

    return char_to_int

In [ ]:
char_to_int = char_mapping(chars)

In [ ]:
char_to_int

{'\n': 0,
 '\r': 1,
 ' ': 2,
 '!': 3,
 '&': 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '?': 22,
 '_': 23,
 'a': 24,
 'b': 25,
 'c': 26,
 'd': 27,
 'e': 28,
 'f': 29,
 'g': 30,
 'h': 31,
 'i': 32,
 'j': 33,
 'k': 34,
 'l': 35,
 'm': 36,
 'n': 37,
 'o': 38,
 'p': 39,
 'q': 40,
 'r': 41,
 's': 42,
 't': 43,
 'u': 44,
 'v': 45,
 'w': 46,
 'x': 47,
 'y': 48,
 'z': 49,
 '£': 50,
 '½': 51,
 'à': 52,
 'â': 53,
 'æ': 54,
 'è': 55,
 'é': 56,
 'œ': 57,
 '—': 58,
 '‘': 59,
 '’': 60,
 '“': 61,
 '”': 62}

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)

print("Total Characters: ", n_chars)
print("Total Unique Vocab: ", n_vocab)

Total Characters:  573498
Total Unique Vocab:  63


**<h4>Question 3:** Create input and output variables.

<details>

**<summary>Hint:</summary>**

 - **Split** the book text up into **subsequences** with a fixed length of **100 characters**, an arbitrary length, which is specified by **maxlen**.

 - Each training pattern of the network should be comprised of **100 time steps of one character (X) followed by one character output (y).** When creating these sequences, slide this window along the whole book one character at a time, allowing each character a chance to be learned from the 100 characters that preceded it. The sliding window movement is specified by **step**.

 - For example, if the sequence length is 5 (for simplicity) then the first two training patterns would be as follows:

  CHAPT -> E <br/>
  
  HAPTE -> R

- Append each sequence to the **sentences** list and each next character to the **next_chars** list.

</details>

In [ ]:
maxlen = 100
step = 1
sentences = []
next_chars = []

In [ ]:
def input_output(raw_text, maxlen, step):
    for i in range(0, len(raw_text) - maxlen, step):
        sentences.append(raw_text[i: i + maxlen])
        next_chars.append(raw_text[i + maxlen])
    
    return sentences, next_chars

In [ ]:
sentences, next_chars = input_output(raw_text, maxlen, step)

In [ ]:
print('Number of sequences:', len(sentences))

Number of sequences: 573398


In [ ]:
sentences[:10]

['i. a scandal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nme',
 '. a scandal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmen',
 ' a scandal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nment',
 'a scandal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmenti',
 ' scandal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmentio',
 'scandal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmention',
 'candal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmention ',
 'andal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmention h',
 'ndal in bohemia\r\n\r\nto sherlock holmes she is always _the_ woman. i have seldom heard him\r\nmention he',
 

In [ ]:
next_chars[:10]

['n', 't', 'i', 'o', 'n', ' ', 'h', 'e', 'r', ' ']

In [ ]:
seq_length = 100
dataX = []
dataY = []

**<h4>Question 4:** Complete the function to convert the characters to integers using **char_to_int**.

<details>

**<summary>Hint:</summary>**

 - Append the integer version of the sequences to **dataX** list.

 - Append the integer version of the next character to **dataY** list.
</details>

In [ ]:
def char_to_int_fn(n_chars, seq_length, raw_text, char_to_int):
    for i in range(0, n_chars - seq_length, 1):
        seq_in = raw_text[i:i + seq_length]
        seq_out = raw_text[i + seq_length]
        dataX.append([char_to_int[char] for char in seq_in])
        dataY.append(char_to_int[seq_out])

    return dataX, dataY

In [ ]:
dataX, dataY = char_to_int_fn(n_chars, seq_length, raw_text, char_to_int)

In [ ]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  573398


In [ ]:
dataX[:2]

[[32,
  9,
  2,
  24,
  2,
  42,
  26,
  24,
  37,
  27,
  24,
  35,
  2,
  32,
  37,
  2,
  25,
  38,
  31,
  28,
  36,
  32,
  24,
  1,
  0,
  1,
  0,
  43,
  38,
  2,
  42,
  31,
  28,
  41,
  35,
  38,
  26,
  34,
  2,
  31,
  38,
  35,
  36,
  28,
  42,
  2,
  42,
  31,
  28,
  2,
  32,
  42,
  2,
  24,
  35,
  46,
  24,
  48,
  42,
  2,
  23,
  43,
  31,
  28,
  23,
  2,
  46,
  38,
  36,
  24,
  37,
  9,
  2,
  32,
  2,
  31,
  24,
  45,
  28,
  2,
  42,
  28,
  35,
  27,
  38,
  36,
  2,
  31,
  28,
  24,
  41,
  27,
  2,
  31,
  32,
  36,
  1,
  0,
  36,
  28],
 [9,
  2,
  24,
  2,
  42,
  26,
  24,
  37,
  27,
  24,
  35,
  2,
  32,
  37,
  2,
  25,
  38,
  31,
  28,
  36,
  32,
  24,
  1,
  0,
  1,
  0,
  43,
  38,
  2,
  42,
  31,
  28,
  41,
  35,
  38,
  26,
  34,
  2,
  31,
  38,
  35,
  36,
  28,
  42,
  2,
  42,
  31,
  28,
  2,
  32,
  42,
  2,
  24,
  35,
  46,
  24,
  48,
  42,
  2,
  23,
  43,
  31,
  28,
  23,
  2,
  46,
  38,
  36,
  24,
  37,
  9,
  2,
  32,
  2

In [ ]:
dataY[:2]

[37, 43]

 - Now that we have **prepared our training data we need to transform it so that it is suitable for use with Keras.**


 - First we must **transform the list of input sequences into the form [samples, time steps, features] expected by an LSTM network.**

In [ ]:
# Reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))

In [ ]:
X.shape

(573398, 100, 1)

In [ ]:
X[0]

array([[32],
       [ 9],
       [ 2],
       [24],
       [ 2],
       [42],
       [26],
       [24],
       [37],
       [27],
       [24],
       [35],
       [ 2],
       [32],
       [37],
       [ 2],
       [25],
       [38],
       [31],
       [28],
       [36],
       [32],
       [24],
       [ 1],
       [ 0],
       [ 1],
       [ 0],
       [43],
       [38],
       [ 2],
       [42],
       [31],
       [28],
       [41],
       [35],
       [38],
       [26],
       [34],
       [ 2],
       [31],
       [38],
       [35],
       [36],
       [28],
       [42],
       [ 2],
       [42],
       [31],
       [28],
       [ 2],
       [32],
       [42],
       [ 2],
       [24],
       [35],
       [46],
       [24],
       [48],
       [42],
       [ 2],
       [23],
       [43],
       [31],
       [28],
       [23],
       [ 2],
       [46],
       [38],
       [36],
       [24],
       [37],
       [ 9],
       [ 2],
       [32],
       [ 2],
       [31],
       [24],

 - Next we need to rescale the integers to the **range 0-to-1** to make the patterns easier to learn by the LSTM network that uses the **sigmoid activation function** by default.

In [ ]:
# Normalizing the data.
X = X / float(n_vocab)

In [ ]:
X[0]

array([[0.50793651],
       [0.14285714],
       [0.03174603],
       [0.38095238],
       [0.03174603],
       [0.66666667],
       [0.41269841],
       [0.38095238],
       [0.58730159],
       [0.42857143],
       [0.38095238],
       [0.55555556],
       [0.03174603],
       [0.50793651],
       [0.58730159],
       [0.03174603],
       [0.3968254 ],
       [0.6031746 ],
       [0.49206349],
       [0.44444444],
       [0.57142857],
       [0.50793651],
       [0.38095238],
       [0.01587302],
       [0.        ],
       [0.01587302],
       [0.        ],
       [0.68253968],
       [0.6031746 ],
       [0.03174603],
       [0.66666667],
       [0.49206349],
       [0.44444444],
       [0.65079365],
       [0.55555556],
       [0.6031746 ],
       [0.41269841],
       [0.53968254],
       [0.03174603],
       [0.49206349],
       [0.6031746 ],
       [0.55555556],
       [0.57142857],
       [0.44444444],
       [0.66666667],
       [0.03174603],
       [0.66666667],
       [0.492

- Finally, we need to convert the **output patterns (single characters converted to integers) into a one hot encoding.** 

- This is so that we can configure the network to predict the probability of each of the 63 different characters in the vocabulary (an easier representation) rather than trying to force it to predict precisely the next character.

- Each **y value is converted into a sparse vector with a length of 63, full of zeros except with a 1 in the column for the letter (integer) that the pattern represents.**

In [ ]:
# One hot encode the output variable
y = to_categorical(dataY)

In [ ]:
y.shape

(573398, 63)

In [ ]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

---
<a name = Section5></a>
# **5. Building LSTM Model**
---

**<h4>Question 5:** Complete the function to build the LSTM model.

<details>

**<summary>Hint:</summary>**
- Define a single **LSTM** layer with **256** hidden units, specify `input_shape` equal to (X.shape[1], X.shape[2]).

- The network should have a **Dropout** layer with dropout rate equal to **0.2** 
 
- The last layer of the network should be a **Dense** layer with hidden units equal to the number of classes to be predicted (i.e. equal to **y.shape[1]**), and an appropriate **activation** function for the last layer of a  **multiclass classification** problem.
</details>

In [ ]:
def build_model():
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))

    return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 63)                16191     
Total params: 280,383
Trainable params: 280,383
Non-trainable params: 0
_________________________________________________________________


**<h4>Question 6:** Complete the function to compile the model.

<details>

**<summary>Hint:</summary>**

- Use appropriate `loss` for a multiclass classification problem, with one-hot encoded target variable.

- Use **adam** as the `optimizer`.
</details>

In [ ]:
def compile_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [ ]:
model = compile_model(model)

**<h4>Question 7:** Create Model Checkpoint callback object.

<details>

**<summary>Hint:</summary>**

- Use the following values for different parameters in the **ModelCheckpoint** class initiation:

  - `filepath`: filepath

  - `monitor`: 'loss'

  - `save_best_only`: True
  
  - `mode`: 'min'

  - `save_weights_only`: True
  </details>

In [ ]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"

In [ ]:
def create_checkpoint(filepath):
    checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True, mode='min')

    return checkpoint

In [ ]:
checkpoint = create_checkpoint(filepath)

In [ ]:
callbacks_list = [checkpoint]

**<h4>Question 8:** Complete the function to train the Model

<details>

**<summary>Hint:</summary>**

- Fit the model on **X** and **y**.

- Specify:

  - `epochs`: 20

  - `batch_size`: 256

  - `callbacks`: callbacks_list

</details>

In [ ]:
def fit_model(model):
    model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

    return model

In [ ]:
model = fit_model(model)

Epoch 1/20
4480/4480 [==============================] - 164s 35ms/step - loss: 2.8202
Epoch 2/20
4480/4480 [==============================] - 157s 35ms/step - loss: 2.5938
Epoch 3/20
4480/4480 [==============================] - 157s 35ms/step - loss: 2.4989
Epoch 4/20
4480/4480 [==============================] - 157s 35ms/step - loss: 2.4174
Epoch 5/20
4480/4480 [==============================] - 155s 35ms/step - loss: 2.3531
Epoch 6/20
4480/4480 [==============================] - 153s 34ms/step - loss: 2.2947
Epoch 7/20
4480/4480 [==============================] - 153s 34ms/step - loss: 2.2438
Epoch 8/20
4480/4480 [==============================] - 155s 35ms/step - loss: 2.1996
Epoch 9/20
4480/4480 [==============================] - 157s 35ms/step - loss: 2.1597
Epoch 10/20
4480/4480 [==============================] - 158s 35ms/step - loss: 2.1266
Epoch 11/20
4480/4480 [==============================] - 158s 35ms/step - loss: 2.1009
Epoch 12/20
4480/4480 [=============================

In [ ]:
# To see the list of all weight checkpoint files created.
!ls

sample_data			    weights-improvement-11-2.1009.hdf5
weights-improvement-01-2.8202.hdf5  weights-improvement-12-2.0717.hdf5
weights-improvement-02-2.5938.hdf5  weights-improvement-13-2.0521.hdf5
weights-improvement-03-2.4989.hdf5  weights-improvement-14-2.0273.hdf5
weights-improvement-04-2.4174.hdf5  weights-improvement-15-2.0076.hdf5
weights-improvement-05-2.3531.hdf5  weights-improvement-16-1.9900.hdf5
weights-improvement-06-2.2947.hdf5  weights-improvement-17-1.9726.hdf5
weights-improvement-07-2.2438.hdf5  weights-improvement-18-1.9580.hdf5
weights-improvement-08-2.1996.hdf5  weights-improvement-19-1.9446.hdf5
weights-improvement-09-2.1597.hdf5  weights-improvement-20-1.9322.hdf5
weights-improvement-10-2.1266.hdf5


**Observations:**

- After running the above code, you should have a number of weight checkpoint files in the local directory.

- You can delete them all except the one with the smallest loss value.

---
<a name = Section6></a>
# **6. Generating Text with LSTM Network**
---

**<h4>Question 1:** Complete the function to load the model

<details>

**<summary>Hint:</summary>**

- Replace the value of **filename** with the model weights having the least loss value during training.

- Use model's `load_weights` method to load the saved model weights.

- Specify **filename** as the model weights to be loaded.
</details>

In [ ]:
# Use the weight checkpoint file that has least loss value.
filename = 'weights-improvement-20-1.9442.hdf5'

In [ ]:
def load_model(model, filename):
    model.load_weights(filename)

    return model

In [ ]:
model = load_model(model, filename)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

**<h4>Question 10:** Create reverse mapping of integers to characters.

<details>

**<summary>Hint:</summary>**

- Enumerate through **`chars`** and store it as a dictionary.

</details>

In [ ]:
def int_to_char_fn(chars):
    int_to_char = dict((i, c) for i, c in enumerate(chars))
    
    return int_to_char

In [ ]:
int_to_char = int_to_char_fn(chars)

In [ ]:
int_to_char

{0: '\n',
 1: '\r',
 2: ' ',
 3: '!',
 4: '&',
 5: '(',
 6: ')',
 7: ',',
 8: '-',
 9: '.',
 10: '0',
 11: '1',
 12: '2',
 13: '3',
 14: '4',
 15: '5',
 16: '6',
 17: '7',
 18: '8',
 19: '9',
 20: ':',
 21: ';',
 22: '?',
 23: '_',
 24: 'a',
 25: 'b',
 26: 'c',
 27: 'd',
 28: 'e',
 29: 'f',
 30: 'g',
 31: 'h',
 32: 'i',
 33: 'j',
 34: 'k',
 35: 'l',
 36: 'm',
 37: 'n',
 38: 'o',
 39: 'p',
 40: 'q',
 41: 'r',
 42: 's',
 43: 't',
 44: 'u',
 45: 'v',
 46: 'w',
 47: 'x',
 48: 'y',
 49: 'z',
 50: '£',
 51: '½',
 52: 'à',
 53: 'â',
 54: 'æ',
 55: 'è',
 56: 'é',
 57: 'œ',
 58: '—',
 59: '‘',
 60: '’',
 61: '“',
 62: '”'}

 - We will use this mapping to convert the integers back to characters so that we can understand the predictions.

 - Finally, we need to actually make predictions.

 - We can pick a random input pattern as our seed sequence, then print generated characters as we generate them.

In [ ]:
# Pick a random seed
start = np.random.randint(0, len(dataX)-1)

In [ ]:
pattern = dataX[start]

In [ ]:
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" s him as
either an innocent man, or else as a man of considerable self-restraint
and firmness. as  "


In [ ]:
# Generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\n\nDone.")


and the mant of the correr of the seale of the coorert which hed hore
the sealed oo the saale and the wert oore the saale and the coorert
shitt of the correr of the saale and the coorer of the sealed waids
fnrm the same oaner of the saale and the coorer of the wert sore, and


seieht of the coorert which hed boeneed and sas an ond oane on the
sererler of the saale and taake and taaken hnr she saale and taedie
the sooe and a sereoner of the wert sorene oane on the saale and the
sooe and taake and taaken hnr she saale and a sereoner of the wert
coean and aaak and a seryere oa lent a sirgo of saaee and taake and

the same was a seale and taake and a seryered aadk to the siae of the
sooe and taake and taaken hnr sie saale and a

Done.


---
<a name = Section7></a>
# **7. Building Large LSTM Network**
---

**<h4>Question 11:** Complete the function to build a larger LSTM model.

<details>

**<summary>Hint:</summary>**

 - Define a network with two **LSTM** layer with **256** hidden units.
 
 - Specify `input_shape` equal to **(X.shape[1], X.shape[2])**, and `return_sequences` equal to **True**, in the first **LSTM** layer.

 - The network should have two **Dropout** layers with dropout rate equal to **0.2**, one after each **LSTM** layer. 
 
- The last layer of the network should be a **Dense** layer with hidden units equal to the number of classes to be predicted (i.e. equal to **y.shape[1]**), and an appropriate **activation** function for the last layer of a  **multiclass classification** problem.
</details>

In [ ]:
def build_bigger_model():
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(256))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))

    return model

In [ ]:
model = build_bigger_model()

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 63)                16191     
Total params: 805,695
Trainable params: 805,695
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

**<h4>Question 12:** Complete the function to train the Model.

<details>

**<summary>Hint:</summary>**

- Fit the model on **X** and **y**.

- Specify:

  - `epochs`: 50

  - `batch_size`: 512

  - `callbacks`: callbacks_list
</details>

In [ ]:
def fit_bigger_model(model):
    model.fit(X, y, epochs=50, batch_size=512, callbacks=callbacks_list)

    return model

In [ ]:
model = fit_bigger_model(model)

Epoch 1/50
1120/1120 [==============================] - 236s 208ms/step - loss: 2.8323

Epoch 00001: loss improved from inf to 2.83228, saving model to weights-improvement-01-2.8323-bigger.hdf5
Epoch 2/50
1120/1120 [==============================] - 232s 207ms/step - loss: 2.4805

Epoch 00002: loss improved from 2.83228 to 2.48050, saving model to weights-improvement-02-2.4805-bigger.hdf5
Epoch 3/50
1120/1120 [==============================] - 233s 208ms/step - loss: 2.3055

Epoch 00003: loss improved from 2.48050 to 2.30547, saving model to weights-improvement-03-2.3055-bigger.hdf5
Epoch 4/50
1120/1120 [==============================] - 233s 208ms/step - loss: 2.1901

Epoch 00004: loss improved from 2.30547 to 2.19012, saving model to weights-improvement-04-2.1901-bigger.hdf5
Epoch 5/50
1120/1120 [==============================] - 233s 208ms/step - loss: 2.1009

Epoch 00005: loss improved from 2.19012 to 2.10088, saving model to weights-improvement-05-2.1009-bigger.hdf5
Epoch 6/50
112

In [ ]:
!ls

---
<a name = Section8></a>
# **8. Generating Text**
---

**<h4>Question 13:** Complete the function to load the model.

<details>

**<summary>Hint:</summary>**

- Replace the value of **filename** with the model weights having the least loss value during training.

- Use model's `load_weights` method to load the saved model weights.

- Specify **filename** as the model weights to be loaded.
</details>

In [ ]:
# Use the weight checkpoint file that has least loss value.
filename = 'weights-improvement-50-1.4850-bigger.hdf5'

In [ ]:
def load_bigger_model(model, filename):
    model.load_weights(filename)

    return model

In [ ]:
model = load_bigger_model(model, filename)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Pick a random seed
start = np.random.randint(0, len(dataX)-1)

In [ ]:
pattern = dataX[start]

In [ ]:
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

In [ ]:
# Generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print("\n\nDone.")

---
<a name = Section9></a>
# **9. Conclusion**
---

- From the above implementations we get an idea bout generating text using LSTM models.

- We learned how to **pre-process** the raw text data into a suitable format.

- Using the processed text we build our LSTM model to generate text. 
- We also learned to save our model and load our model.

- You can further try experimenting with different model architecture and try to improve the performance of the model.